<a href="https://colab.research.google.com/github/Estefany66/Ciencia-de-Datos/blob/main/REPORTES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Librerías utilizadas
!pip install squarify

# Librerías para el manejo de directorios
import os
import glob
from pathlib import Path

import numpy as np
import pandas as pd

# Librerías para graficar
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import squarify

%matplotlib inline

In [7]:
# Carga de datos

BASE_DIR = Path.cwd()
BASE_DIR
df_ventas= pd.read_excel('//content/drive/MyDrive/DATA OPTICA SAN FRANCISCO /Producto_Venta.xlsx')

df_stock = pd.read_excel('///content/drive/MyDrive/DATA OPTICA SAN FRANCISCO /Producto_Stock.xlsx')

df_ventas_linea_prod = df_ventas.query('fecha > "2022-08-01"').loc[:,['fecha_comp', 'cod', 'producto', 'cantidad', 'monto_soles']]
df_ventas_linea_prod = pd.merge(df_ventas_linea_prod, df_stock, on=['cod', 'producto'], how='outer')

In [8]:
# Estilo y colores para los gráficos

plt.style.use('seaborn-colorblind')
plt.rcParams['lines.linewidth'] = 1.5

VERDE = '#39894d'
AZUL = '#2649e8'
AMARILLO = '#f1d065'

# Funciones generales

# Formateado de texto
def formato_titulo(texto):
    subray = '=' * (len(texto))
    print("\033[1;32m"+texto)
    print(subray)
    print("\033[0;m")

<ipython-input-8-05e5a8c5b220>:3: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-colorblind')


In [9]:
# Funciones para crear gráficos de Altair


# graf_barras_ventas_dol
# Crea un gráfico de barras de ventas en $ en función de diferentes
# intervalos de tiempo
#
# Parámetros:
#     df: DataFrame a utilizar
#     x: Datos para el eje de las x. Generalmente es el intervalo de tiempo
#     tooltip: Lista con los parámetros del tooltip
#     titulo: Título del gráfico
#     dx: Entero para ajustar horizontalmente la posición del texto sobre las barras
def graf_barras_ventas_dol(df, x, tooltip, titulo, dx):


    barras = alt.Chart(df).mark_bar().encode(
                    x=x,
                    y=alt.Y('sum(monto_soles):Q', title = 'Ventas en S/.', scale=alt.Scale(zero=False),
                      axis = alt.Axis(grid=True, titleAnchor='middle', titleAngle = 270, labelFontSize=10)),
                    color=alt.Color(
                        'sum(monto_soles):Q', scale=alt.Scale(scheme='goldgreen'),
                        legend=alt.Legend(title="Ventas en S/.")),
                    tooltip=tooltip,
                    ).properties(title=titulo,
                             width=800,
                             height=300
                    )

    textos = barras.mark_text(dx=dx, dy=-10, fill='black').encode(
             text = alt.Text('sum(monto_soles):Q', format=',.5', title = 'Ventas en S/.')
                    )

    grafico_final = barras + textos

    return grafico_final

# graf_sectores_linea
# Crea un gráfico de sectores de las líneas de productos
#
# Parámetros:
#     df: DataFrame a utilizar
#     theta: Datos que se van a graficar
#     texto: Texto que se mostrará en el cada sector del gráfico
#     titulo: Título del gráfico
def graf_sectores_linea(df, theta, texto, titulo):

    base = alt.Chart(df).encode(
        theta=theta,
        color=alt.Color("linea:N"),
        tooltip=[alt.Tooltip('linea:N', title='Línea de Producto'),
                 alt.Tooltip('sum(monto_soles):Q', format='$,.5', title='Ventas en S/.'),
                 alt.Tooltip('sum(cantidad):Q', format=',.5', title='Volumen de Ventas')]
    ).properties(title=titulo)

    pie = base.mark_arc(outerRadius=130, innerRadius=40)

    text = base.mark_text(radius=110, size=10, fill='black').encode(
           text=texto
           )

    grafico = pie + text

    return grafico

In [14]:
# Agrupar por 'fecha' y sumar la columna 'monto_dolar'
df_ventas_sol = df_ventas.groupby('fecha')['monto_soles'].sum().reset_index()

# Convertir la columna 'fecha' a tipo datetime
df_ventas_sol['fecha'] = pd.to_datetime(df_ventas_sol['fecha'])

# Calcular estadísticas adicionales
total_dias = df_ventas_sol['fecha'].nunique()
num_meses = df_ventas_sol['fecha'].dt.strftime('%m-%Y').nunique()
max_ventas = df_ventas_sol['monto_soles'].max()


<ipython-input-14-12b2dfd99c69>:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_ventas_sol['fecha'] = pd.to_datetime(df_ventas_sol['fecha'])


In [16]:
# Agrupar por 'fecha' y sumar la columna 'monto_dolar'
df_ventas_sol = df_ventas.groupby('fecha')['monto_soles'].sum().reset_index()

# Convertir la columna 'fecha' a tipo datetime
df_ventas_sol['fecha'] = pd.to_datetime(df_ventas_sol['fecha'], errors='coerce')

# Filtrar filas con fechas no válidas (si es necesario)
df_ventas_sol = df_ventas_sol.dropna(subset=['fecha'])

# Calcular estadísticas adicionales
total_dias = df_ventas_sol['fecha'].nunique()
num_meses = df_ventas_sol['fecha'].dt.strftime('%m-%Y').nunique()
max_ventas = df_ventas_sol['monto_soles'].max()


<ipython-input-16-f460a5e06fef>:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_ventas_sol['fecha'] = pd.to_datetime(df_ventas_sol['fecha'], errors='coerce')


In [19]:
# Agrupar por 'fecha' y sumar la columna 'monto_dolar'
df_ventas_sol = df_ventas.groupby('fecha')['monto_soles'].sum().reset_index()

# Convertir la columna 'fecha' a tipo datetime
df_ventas_sol['fecha'] = pd.to_datetime(df_ventas_sol['fecha'], errors='coerce')

# Filtrar filas con fechas no válidas (si es necesario)
df_ventas_sol = df_ventas_sol.dropna(subset=['fecha'])

# Calcular el número de meses
num_meses = df_ventas_sol['fecha'].dt.strftime('%m-%Y').nunique()


<ipython-input-19-a4919d13a6a7>:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_ventas_sol['fecha'] = pd.to_datetime(df_ventas_sol['fecha'], errors='coerce')


In [37]:
# Resumen General de las Ventas en $

# Agrupar por 'fecha' y sumar la columna 'monto_dolar'
df_ventas_sol = df_ventas.groupby('fecha_comp')['monto_soles'].sum().reset_index()

df_ventas_fecha = df_ventas.pivot_table(index='fecha_comp',
                                       aggfunc={'monto_soles': sum,
                                                'cantidad': sum,
                                                'num': 'nunique'})
total_dias = df_ventas_sol['fecha_comp'].nunique()
num_meses = df_ventas_sol['fecha_comp'].nunique()
max_ventas = df_ventas_sol['monto_soles'].max()


total_ventas_sol = df_ventas_sol.monto_soles.sum()
total_faturas = df_ventas_fecha.num.sum()
promedio_diario =  total_ventas_sol / total_dias
promedio_mensual = total_ventas_sol / num_meses

formato_titulo('Resumen General de las Ventas en $')

print('El total de ventas en $ del período es:', round(total_ventas_sol, 2))
print('El N° de Facturas emitidas durante el período es:', round(total_faturas, 2))
print('El promedio mensual de ventas en $ es:', round(promedio_mensual, 2))
print('El promedio diario es:', round(promedio_diario, 2))

Resumen General de las Ventas en $

El total de ventas en $ del período es: 890033.37
El N° de Facturas emitidas durante el período es: 4877
El promedio mensual de ventas en $ es: 1692.08
El promedio diario es: 1692.08
